In [44]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [45]:
!pip install stable-baselines3[extra]

Defaulting to user installation because normal site-packages is not writeable


In [46]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [47]:
environment_name = "CartPole-v0"

In [48]:
env = gym.make(environment_name)

In [49]:
episodes = 5 f
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:15.0
Episode:2 Score:35.0
Episode:3 Score:45.0
Episode:4 Score:39.0
Episode:5 Score:12.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [50]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [51]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([-3.098262e+00,  3.328287e+37,  2.164019e-01,  6.116359e+37],
      dtype=float32)

# 3. Train an RL Model

In [52]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log = "./Logs/")


Using cpu device


In [53]:
model.learn(total_timesteps=20000)

2021-09-18 13:15:07.799066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-18 13:15:07.799121: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Logging to ./Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 168  |
|    iterations      | 1    |
|    time_elapsed    | 12   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 274         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008000955 |
|    clip_fraction        | 0.075       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.00448     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.05        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0114     |
|    value_loss           | 54          |
-----------------------------------------
----------

# 4. Save and Reload Model

In [54]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [55]:
model.save(PPO_path)

In [56]:
#del model

In [57]:
#model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [58]:
from stable_baselines3.common.evaluation import evaluate_policy

In [59]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/ziad/.local/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [60]:
env.close()

# 5. Test Model

In [61]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([0.53564685, 0.01510247, 0.02941947, 0.48786682])}]


In [62]:
env.close()

# 6. Viewing Logs in Tensorboard

In [65]:
training_log_path = os.path.join("./Logs/", 'PPO_3')
training_log_path

'./Logs/PPO_3'

In [ ]:
#!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()